In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# 시각화 한글폰트 설정
# 맑은 고딕

plt.rc('font', family='Malgun Gothic')
sns.set(font="Malgun Gothic",#"NanumGothicCoding", 
        rc={"axes.unicode_minus":False}, # 마이너스 부호 깨짐 현상 해결
        style='darkgrid')

# scipy
import scipy.stats as spst

In [30]:
wbc_p = pd.read_csv('./국대_투수.csv', encoding = 'cp949')
wbc_b = pd.read_csv('./국대_타자.csv', encoding = 'cp949')
kbo_p = pd.read_csv('./kbo_투수2.csv', encoding = 'cp949')
kbo_b = pd.read_csv('./kbo_타자2.csv', encoding = 'cp949')
wbc_won_p = pd.read_csv('./원장희_국대_투수.csv', encoding = 'cp949')
wbc_won_b = pd.read_csv('./원장희_국대_타자.csv', encoding = 'cp949')

In [31]:
kbo_b.replace('kt','KT', inplace=True)
kbo_b.replace('SK','SSG', inplace=True)
kbo_b.replace('넥센','키움', inplace=True)
kbo_p.replace('kt','KT', inplace=True)
kbo_p.replace('SK','SSG', inplace=True)
kbo_p.replace('넥센','키움', inplace=True)


wbc_b.rename(columns = {'WAR*' : '타자WAR', 'WPA' : '타자WPA'}, inplace = True)
wbc_p.rename(columns = {'WAR' : '투수WAR', 'WPA' : '투수WPA'}, inplace = True)

# 타자 세이버 지표 추가

In [32]:
#GPA 지표 추가
kbo_b['GPA'] = (kbo_b['출루']*1.8 + kbo_b['장타'])/4
wbc_b['GPA'] = (wbc_b['출루']*1.8 + wbc_b['장타'])/4
wbc_won_b['GPA'] = (wbc_won_b['출루']*1.8 + wbc_won_b['장타'])/4

#XR 지표 추가
kbo_b['XR'] = (kbo_b['안타']-kbo_b['2타']-kbo_b['3타']-kbo_b['홈런'])*0.5 + kbo_b['2타']*0.72 + kbo_b['3타']*1.04 + kbo_b['홈런']*1.44 + (kbo_b['볼넷']-kbo_b['사구'])*0.34 + kbo_b['고4']*0.25 + kbo_b['도루']*0.18 -  kbo_b['도실']*0.32 - (kbo_b['타수']-kbo_b['안타']-kbo_b['삼진'])*0.09 - kbo_b['삼진']*0.098 - kbo_b['병살']*0.37 + kbo_b['희비']*0.37 + kbo_b['희타']*0.04
wbc_b['XR'] = (wbc_b['안타']-wbc_b['2타']-wbc_b['3타']-wbc_b['홈런'])*0.5 + wbc_b['2타']*0.72 + wbc_b['3타']*1.04 + wbc_b['홈런']*1.44 + (wbc_b['볼넷']-wbc_b['사구'])*0.34 + wbc_b['고4']*0.25 + wbc_b['도루']*0.18 -  wbc_b['도실']*0.32 - (wbc_b['타수']-wbc_b['안타']-wbc_b['삼진'])*0.09 - wbc_b['삼진']*0.098 - wbc_b['병살']*0.37 + wbc_b['희비']*0.37 + wbc_b['희타']*0.04
wbc_won_b['XR'] = (wbc_won_b['안타']-wbc_won_b['2타']-wbc_won_b['3타']-wbc_won_b['홈런'])*0.5 + wbc_won_b['2타']*0.72 + wbc_won_b['3타']*1.04 + wbc_won_b['홈런']*1.44 + (wbc_won_b['볼넷']-wbc_won_b['사구'])*0.34 + wbc_won_b['고4']*0.25 + wbc_won_b['도루']*0.18 -  wbc_won_b['도실']*0.32 - (wbc_won_b['타수']-wbc_won_b['안타']-wbc_won_b['삼진'])*0.09 - wbc_won_b['삼진']*0.098 - wbc_won_b['병살']*0.37 + wbc_won_b['희비']*0.37 + wbc_won_b['희타']*0.04

#XR27 지표 추가
kbo_b['XR27'] = (kbo_b['XR']/(kbo_b['타석']-kbo_b['안타']+kbo_b['희타']+kbo_b['희비']+kbo_b['도실']+kbo_b['병살'])) * 27
wbc_b['XR27'] = (wbc_b['XR']/(wbc_b['타석']-wbc_b['안타']+wbc_b['희타']+wbc_b['희비']+wbc_b['도실']+wbc_b['병살'])) * 27
wbc_won_b['XR27'] = (wbc_won_b['XR']/(wbc_won_b['타석']-wbc_won_b['안타']+wbc_won_b['희타']+wbc_won_b['희비']+wbc_won_b['도실']+wbc_won_b['병살'])) * 27

#RC27 지표 추가
#RC = A*B/C  A = 안타+볼넷+사구-도실-병살, B = 루타 + 0.26(볼넷-고4+사구) + 0.52*(희타+희비+도루), C = 타석 + 볼넷 + 사구 + 희타 + 희비 
#RC27 = RC/(타석-안타+희타+희비+도실+병살)*27

kbo_b['A'] = kbo_b['안타'] + kbo_b['볼넷'] + kbo_b['사구'] - kbo_b['도실'] - kbo_b['병살']
kbo_b['B'] = kbo_b['루타'] + 0.26*(kbo_b['볼넷'] - kbo_b['고4'] + kbo_b['사구']) + 0.52*(kbo_b['희타'] + kbo_b['희비'] + kbo_b['도루'])
kbo_b['C'] = kbo_b['타석'] + kbo_b['볼넷'] + kbo_b['사구'] + kbo_b['희타'] + kbo_b['희비']
kbo_b['RC'] = kbo_b['A']*kbo_b['B']/kbo_b['C']
kbo_b['RC27'] = (kbo_b['RC']/(kbo_b['타석'] - kbo_b['안타'] + kbo_b['희타'] + kbo_b['희비'] + kbo_b['도실'] + kbo_b['병살']))*27



wbc_b['A'] = wbc_b['안타'] + wbc_b['볼넷'] + wbc_b['사구'] - wbc_b['도실'] - wbc_b['병살']
wbc_b['B'] = wbc_b['루타'] + 0.26*(wbc_b['볼넷'] - wbc_b['고4'] + wbc_b['사구']) + 0.52*(wbc_b['희타'] + wbc_b['희비'] + wbc_b['도루'])
wbc_b['C'] = wbc_b['타석'] + wbc_b['볼넷'] + wbc_b['사구'] + wbc_b['희타'] + wbc_b['희비']
wbc_b['RC'] = wbc_b['A']*wbc_b['B']/wbc_b['C']
wbc_b['RC27'] = (wbc_b['RC']/(wbc_b['타석'] - wbc_b['안타'] + wbc_b['희타'] + wbc_b['희비'] + wbc_b['도실'] + wbc_b['병살']))*27


wbc_won_b['A'] = wbc_won_b['안타'] + wbc_won_b['볼넷'] + wbc_won_b['사구'] - wbc_won_b['도실'] - wbc_won_b['병살']
wbc_won_b['B'] = wbc_won_b['루타'] + 0.26*(wbc_won_b['볼넷'] - wbc_won_b['고4'] + wbc_won_b['사구']) + 0.52*(wbc_won_b['희타'] + wbc_won_b['희비'] + wbc_won_b['도루'])
wbc_won_b['C'] = wbc_won_b['타석'] + wbc_won_b['볼넷'] + wbc_won_b['사구'] + wbc_won_b['희타'] + wbc_won_b['희비']
wbc_won_b['RC'] = wbc_won_b['A']*wbc_won_b['B']/wbc_won_b['C']
wbc_won_b['RC27'] = (wbc_won_b['RC']/(wbc_won_b['타석'] - wbc_won_b['안타'] + wbc_won_b['희타'] + wbc_won_b['희비'] + wbc_won_b['도실'] + wbc_won_b['병살']))*27




# 데이터 변형

In [33]:
wbc_won_b['팀'] = '국가대표'

In [34]:
wbc_b2 = wbc_b[['팀','OPS', 'wOBA', 'wRC+', '타자WAR', '타자WPA', 'GPA', 'XR27','RC27']]
kbo_b2 = kbo_b[['팀','연도', 'OPS', 'wOBA', 'wRC+', '타자WAR', '타자WPA', 'GPA', 'XR27','RC27']]
wbc_won_b2 = wbc_won_b[['팀', 'OPS', 'wOBA', 'wRC+', '타자WAR', '타자WPA', 'GPA', 'XR27','RC27']]

In [35]:
print(wbc_p['이닝'].sum()) #1844 
print(wbc_b['타석'].sum()) #6970
print(kbo_p['이닝'].mean()) #1277
print(kbo_b['타석'].mean()) #5655 
print(wbc_won_p['이닝'].sum()) #1662
print(wbc_won_b['타석'].sum()) #6958
#WAR, WPA는 합지표라 비율 맞춰줘야함

1843.6999999999998
6970
1277.4524999999999
5655.0875
1661.6000000000001
6958


In [36]:
wbc_p2 = wbc_p[['ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+', '투수WAR', '투수WPA']]
wbc_won_p2 = wbc_won_p[['ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+', '투수WAR', '투수WPA']]
kbo_p2 = kbo_p[['팀', '연도', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+', '투수WAR', '투수WPA']]

In [37]:
#1 단순 비율
wbc_b3 = wbc_b2
wbc_won_b3 = wbc_won_b2

wbc_won_b3['타자WAR'] = wbc_won_b2['타자WAR']*5655/6958
wbc_won_b3['타자WPA'] = wbc_won_b2['타자WPA']*5655/6958
wbc_won_p2['투수WAR'] = wbc_won_p2['투수WAR']*1277/1662
wbc_won_p2['투수WPA'] = wbc_won_p2['투수WPA']*1277/1662

wbc_b3['타자WAR'] = wbc_b2['타자WAR']*5655/6970
wbc_b3['타자WPA'] = wbc_b2['타자WPA']*5655/6970
wbc_p2['투수WAR'] = wbc_p2['투수WAR']*1277/1844
wbc_p2['투수WPA'] = wbc_p2['투수WPA']*1277/1844

C:\Users\User\AppData\Local\Temp\ipykernel_8996\1834189102.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wbc_won_b3['타자WAR'] = wbc_won_b2['타자WAR']*5655/6958
C:\Users\User\AppData\Local\Temp\ipykernel_8996\1834189102.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wbc_won_b3['타자WPA'] = wbc_won_b2['타자WPA']*5655/6958
C:\Users\User\AppData\Local\Temp\ipykernel_8996\1834189102.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [38]:
from collections import defaultdict

kbo_b.replace('kt','KT', inplace=True)
kbo_b.replace('SK','SSG', inplace=True)
kbo_b.replace('넥센','키움', inplace=True)
kbo_p.replace('kt','KT', inplace=True)
kbo_p.replace('SK','SSG', inplace=True)
kbo_p.replace('넥센','키움', inplace=True)

kbo = ['LG', "두산", "키움", "SSG", "한화", "롯데", "삼성", "KT", "KIA", "NC"]

kbo_dict_b = defaultdict()
kbo_dict_p = defaultdict()


for i in range(len(kbo)):
    kbo_dict_b[kbo[i]] = kbo_b2[kbo_b2["팀"] == kbo[i]].sort_values(by = "연도")
    kbo_dict_p[kbo[i]] = kbo_p2[kbo_p2["팀"] == kbo[i]].sort_values(by = "연도")
    
LG_p = kbo_dict_p['LG']
DS_p = kbo_dict_p['두산']
KW_p = kbo_dict_p['키움']
SSG_p = kbo_dict_p['SSG']
HH_p = kbo_dict_p['한화']
LT_p = kbo_dict_p['롯데']
SS_p = kbo_dict_p['삼성']
KT_p = kbo_dict_p['KT']
KIA_p = kbo_dict_p['KIA']
NC_p = kbo_dict_p['NC']

LG_b = kbo_dict_b['LG']
DS_b = kbo_dict_b['두산']
KW_b = kbo_dict_b['키움']
SSG_b = kbo_dict_b['SSG']
HH_b = kbo_dict_b['한화']
LT_b = kbo_dict_b['롯데']
SS_b = kbo_dict_b['삼성']
KT_b = kbo_dict_b['KT']
KIA_b = kbo_dict_b['KIA']
NC_b = kbo_dict_b['NC']

LG = pd.merge(LG_b,LG_p)
DS = pd.merge(DS_b,DS_p)
KW = pd.merge(KW_b,KW_p)
SSG = pd.merge(SSG_b,SSG_p)
HH = pd.merge(HH_b,HH_p)
LT = pd.merge(LT_b,LT_p)
SS = pd.merge(SS_b,SS_p)
KT = pd.merge(KT_b,KT_p)
KIA = pd.merge(KIA_b,KIA_p)
NC = pd.merge(NC_b,NC_p)

LG["승 수"] = [64,71,69,68,79,79,72,87]
DS["승 수"] = [79,93,84,93,88,79,71,60]
KW["승 수"] = [78,77,69,75,86,80,70,80]
SSG["승 수"] = [69,69,75,78,88,51,66,88]
HH["승 수"] = [68,66,61,77,58,46,49,46]
LT["승 수"] = [66,66,80,68,48,71,65,64]
SS["승 수"] = [88,65,55,68,60,64,76,66]
KT["승 수"] = [52,53,50,59,71,81,76,80]
KIA["승 수"] = [67,70,87,70,62,73,58,70]
NC["승 수"] = [84,83,79,58,73,83,67,67] 

LG["승률"] = LG['승 수']/144
DS["승률"] = DS['승 수']/144
KW["승률"] = KW['승 수']/144
SSG["승률"] = SSG['승 수']/144
HH["승률"] = HH['승 수']/144
LT["승률"] = LT['승 수']/144
SS["승률"] = SS['승 수']/144
KT["승률"] = KT['승 수']/144
KIA["승률"] = KIA['승 수']/144
NC["승률"] = NC['승 수']/144

In [39]:
#데이터 세로로 다시 병합
kbo_team = [LG, DS, KW, SSG, HH, LT, SS, KT, KIA, NC]
kbo_re = pd.concat(kbo_team)

In [43]:
wbc = []
wbc.append(wbc_b3['OPS'].mean())
wbc.append(wbc_b3['wOBA'].mean())
wbc.append(wbc_b3['wRC+'].mean())
wbc.append(wbc_b3['타자WAR'].sum())
wbc.append(wbc_b3['타자WPA'].sum())
wbc.append(wbc_b3['GPA'].mean())
wbc.append(wbc_b3['XR27'].mean())
wbc.append(wbc_b3['RC27'].mean())
wbc.append(wbc_p2['ERA'].mean())
wbc.append(wbc_p2['FIP'].mean())
wbc.append(wbc_p2['WHIP'].mean())
wbc.append(wbc_p2['ERA+'].mean())
wbc.append(wbc_p2['FIP+'].mean())
wbc.append(wbc_p2['투수WAR'].sum())
wbc.append(wbc_p2['투수WPA'].sum())
print(wbc)

wbc_won = []
wbc_won.append(wbc_won_b3['OPS'].mean())
wbc_won.append(wbc_won_b3['wOBA'].mean())
wbc_won.append(wbc_won_b3['wRC+'].mean())
wbc_won.append(wbc_won_b3['타자WAR'].sum())
wbc_won.append(wbc_won_b3['타자WPA'].sum())
wbc_won.append(wbc_won_b3['GPA'].mean())
wbc_won.append(wbc_won_b3['XR27'].mean())
wbc_won.append(wbc_won_b3['RC27'].mean())
wbc_won.append(wbc_won_p2['ERA'].mean())
wbc_won.append(wbc_won_p2['FIP'].mean())
wbc_won.append(wbc_won_p2['WHIP'].mean())
wbc_won.append(wbc_won_p2['ERA+'].mean())
wbc_won.append(wbc_won_p2['FIP+'].mean())
wbc_won.append(wbc_won_p2['투수WAR'].sum())
wbc_won.append(wbc_won_p2['투수WPA'].sum())
print(wbc_won)

[0.8234615384615385, 0.3723846153846154, 132.46153846153845, 47.68211621233859, 21.800552367288386, 0.27926923076923077, 4.843267817549409, 4.776920969422313, 3.0880000000000005, 3.3786666666666667, 1.2233333333333334, 143.55333333333334, 123.55999999999999, 29.61892082429501, 14.535916485900218]
[0.8533076923076923, 0.38323076923076915, 139.23846153846154, 49.92622161540673, 25.934327392929006, 0.28758076923076925, 5.1071871226875345, 4.94067774969287, 2.6146666666666665, 3.1799999999999997, 1.162, 166.8, 131.93333333333334, 41.61391095066185, 26.3160348977136]


# 데이터 스케일링

In [45]:
pd.options.mode.chained_assignment = None

def scaling(df, scale_columns):
    for i in scale_columns:
        series_mean = df[i].mean()
        series_std = df[i].std()
        df[i] = df[i].apply(lambda x : (x - series_mean)/series_std)
    return df

In [51]:
kbo_re2 = kbo_re[kbo_re.columns.difference(['팀', '연도', '승 수', '승률'])]

# wbc 스케일링 해주기 위해 병합

In [65]:
wbc_re = pd.DataFrame([wbc, wbc_won], columns = ['OPS', 'wOBA', 'wRC+', '타자WAR', '타자WPA', 'GPA', 'XR27',
       'RC27', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+', '투수WAR', '투수WPA'], dtype = float)

In [68]:
kbo_scale = pd.concat([kbo_re2, wbc_re])

In [69]:
column = wbc_re.columns
scaling(kbo_scale, column)

,ERA,ERA+,FIP,FIP+,GPA,OPS,RC27,WHIP,XR27,wOBA,wRC+,타자WAR,타자WPA,투수WAR,투수WPA
0,-0.050431,-0.084467,-0.279864,0.334121,-0.566055,-0.527144,-0.552397,-0.203053,-0.619116,-0.825677,-0.830933,-0.848955,-0.751404,0.804160,-0.095617
1,0.640622,-0.187781,0.685546,-0.236374,0.373696,0.291538,0.379009,0.665494,0.455861,0.453304,-0.283987,-0.235779,0.699288,-0.531135,-0.747758
2,-0.531163,0.789730,-0.462016,1.103626,-0.285435,-0.327465,-0.434119,-1.361117,-0.625459,-0.651270,-0.747500,-0.781641,-0.401826,0.286944,0.083958
3,0.971126,-0.585144,0.667331,-0.209840,0.595582,0.650960,0.828322,0.279473,0.683941,0.162627,0.133175,0.113035,0.784690,-0.747486,-1.206673
4,-1.177147,0.265212,-0.953829,-0.382315,-1.084876,-1.066276,-0.927620,-0.878591,-1.031447,-1.000083,-0.209825,-0.046073,-0.517972,-0.444932,1.189763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,-0.125545,0.344684,0.394101,-0.249642,1.254713,1.269963,1.345918,-0.396064,1.243164,1.499743,1.282689,1.347956,1.213977,0.344413,0.279285
6,-0.200659,-0.171887,-0.316294,-0.090434,-0.171229,-0.107819,-0.299776,0.182968,-0.406371,0.278898,0.318581,0.197484,-0.294789,-0.135617,0.081858
7,-1.147102,0.161898,-1.154197,0.082042,-1.104454,-1.106212,-1.084377,-0.975096,-1.134157,-1.058219,-0.052230,-0.157449,-1.646415,0.212574,1.039592
0,-2.381983,3.289405,-2.377049,3.022083,1.193970,1.179340,1.621742,-2.294002,1.595010,1.638374,2.910691,3.116759,1.670655,2.054760,2.161770


# kbo 따로 스케일링

In [79]:
scaling(kbo_re, kbo_re.columns.difference(['팀', '연도', '승 수', '승률']))

,팀,연도,OPS,wOBA,wRC+,타자WAR,타자WPA,GPA,XR27,RC27,ERA,FIP,WHIP,ERA+,FIP+,투수WAR,투수WPA,승 수,승률
0,LG,15,-0.498651,-0.808258,-0.864681,-0.886317,-0.729899,-0.538176,-0.592500,-0.523161,-0.130660,-0.371893,-0.290637,0.028308,0.511620,1.013789,-0.028921,64,0.444444
1,LG,16,0.334127,0.522457,-0.234365,-0.178309,0.774688,0.417107,0.519385,0.436019,0.628945,0.672362,0.651970,-0.111883,-0.177484,-0.522992,-0.751279,71,0.493056
2,LG,17,-0.295535,-0.626797,-0.768531,-0.808592,-0.367333,-0.252918,-0.599061,-0.401356,-0.659081,-0.568922,-1.547448,1.214541,1.441109,0.418529,0.169990,69,0.479167
3,LG,18,0.699736,0.220022,0.246384,0.224450,0.863262,0.642660,0.755295,0.898731,0.992234,0.652659,0.233034,-0.651080,-0.145433,-0.771989,-1.259605,68,0.472222
4,LG,19,-1.047066,-0.989719,-0.148899,0.040735,-0.487795,-1.065571,-1.018988,-0.909573,-1.369147,-1.100901,-1.023777,0.502801,-0.353767,-0.423782,1.394858,79,0.548611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,NC,18,-0.600209,-1.292154,-1.772764,-1.677703,-0.080352,-0.753778,-0.711317,-0.543560,1.289471,1.618102,1.070907,-0.705000,-0.834537,-1.172719,-1.482943,58,0.402778
4,NC,19,-0.011171,0.159535,0.801917,0.689389,-0.925944,-0.077119,-0.146350,-0.122894,-1.154476,-0.982683,-0.919043,0.438097,-0.017228,0.074213,1.144766,73,0.506944
5,NC,20,1.329398,1.611223,1.571116,1.650358,1.308497,1.312684,1.333718,1.431762,-0.213226,0.357115,-0.500106,0.610640,-0.193510,0.484669,0.386348,83,0.576389
6,NC,21,-0.072106,0.340996,0.460051,0.321960,-0.256320,-0.136824,-0.372451,-0.263007,-0.295792,-0.411298,0.128299,-0.090315,-0.001202,-0.067793,0.167663,67,0.465278


In [80]:
kbo_re.drop(columns = ['팀', '연도', '승 수'], axis = True, inplace = True)

# Auto ML

In [ ]:
pip install mljar-supervised


In [108]:
from supervised.automl import AutoML

In [120]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

In [127]:
x_train, x_val, y_train, y_val = train_test_split(
    kbo_re[kbo_re.columns.difference(['승률'])],
    kbo_re['승률'],
    test_size=0.2,
    random_state=123,
)

In [148]:
automl = AutoML(mode="Explain")
automl.fit(x_train, y_train)

AutoML directory: AutoML_5
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Baseline', 'Linear', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 3 models
1_Baseline rmse 0.05892 trained in 0.53 seconds


c:\Users\User\anaconda3\lib\site-packages\shap\plots\_beeswarm.py:890: UserWarning: Glyph 53440 (\N{HANGUL SYLLABLE TA}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\shap\plots\_beeswarm.py:890: UserWarning: Glyph 51088 (\N{HANGUL SYLLABLE JA}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\shap\plots\_beeswarm.py:890: UserWarning: Glyph 53804 (\N{HANGUL SYLLABLE TU}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\shap\plots\_beeswarm.py:890: UserWarning: Glyph 49688 (\N{HANGUL SYLLABLE SU}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\shap.py:119: UserWarning: Glyph 53440 (\N{HANGUL SYLLABLE TA}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\shap.py:119: UserWarning: Glyph 51088 (\N{HANGUL SYLLABLE JA}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\shap.py:119: UserWarning: Glyph 53804 (\N{HANGUL SY

2_DecisionTree rmse 0.040016 trained in 9.83 seconds


c:\Users\User\anaconda3\lib\site-packages\supervised\utils\additional_metrics.py:558: UserWarning: Glyph 53440 (\N{HANGUL SYLLABLE TA}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\additional_metrics.py:558: UserWarning: Glyph 51088 (\N{HANGUL SYLLABLE JA}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\additional_metrics.py:558: UserWarning: Glyph 53804 (\N{HANGUL SYLLABLE TU}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\additional_metrics.py:558: UserWarning: Glyph 49688 (\N{HANGUL SYLLABLE SU}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\additional_metrics.py:564: UserWarning: Glyph 53440 (\N{HANGUL SYLLABLE TA}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\additional_metrics.py:564: UserWarning: Glyph 51088 (\N{HANGUL SYLLABLE JA}) missing from current font.
c:\Users\User\anaconda

3_Linear rmse 0.007977 trained in 4.1 seconds


...

c:\Users\User\anaconda3\lib\site-packages\supervised\utils\additional_metrics.py:558: UserWarning: Glyph 53440 (\N{HANGUL SYLLABLE TA}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\additional_metrics.py:558: UserWarning: Glyph 51088 (\N{HANGUL SYLLABLE JA}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\additional_metrics.py:558: UserWarning: Glyph 53804 (\N{HANGUL SYLLABLE TU}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\additional_metrics.py:558: UserWarning: Glyph 49688 (\N{HANGUL SYLLABLE SU}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\additional_metrics.py:564: UserWarning: Glyph 53440 (\N{HANGUL SYLLABLE TA}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\additional_metrics.py:564: UserWarning: Glyph 51088 (\N{HANGUL SYLLABLE JA}) missing from current font.
c:\Users\User\anaconda

* Step ensemble will try to check up to 1 model
Ensemble rmse 0.007977 trained in 0.14 seconds


c:\Users\User\anaconda3\lib\site-packages\supervised\utils\automl_plots.py:106: UserWarning: Glyph 53440 (\N{HANGUL SYLLABLE TA}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\automl_plots.py:106: UserWarning: Glyph 51088 (\N{HANGUL SYLLABLE JA}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\automl_plots.py:106: UserWarning: Glyph 53804 (\N{HANGUL SYLLABLE TU}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\automl_plots.py:106: UserWarning: Glyph 49688 (\N{HANGUL SYLLABLE SU}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\automl_plots.py:108: UserWarning: Glyph 53440 (\N{HANGUL SYLLABLE TA}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils\automl_plots.py:108: UserWarning: Glyph 51088 (\N{HANGUL SYLLABLE JA}) missing from current font.
c:\Users\User\anaconda3\lib\site-packages\supervised\utils

AutoML fit time: 21.92 seconds
AutoML best model: 3_Linear


AutoML()

In [149]:
y_pred = automl.predict(x_val)
print(mean_absolute_error(y_val, y_pred))
print(r2_score(y_val, y_pred))

0.009412675378481776
0.9832398291239481


In [150]:
y_pred

array([0.46445157, 0.34432112, 0.42481998, 0.54721306, 0.49449782,
       0.44002458, 0.60926329, 0.58563561, 0.5389471 , 0.4812285 ,
       0.64277663, 0.29574755, 0.38798856, 0.32384451, 0.58833545,
       0.45839649])

In [151]:
x_test = kbo_scale.iloc[80:]

In [152]:
automl.predict(x_test)

array([0.72211817, 0.82562814])